The Bickley jet is an idealized model of geophysical flows such as the Gulf Stream  and models a meandering zonal jet between a series of counter rotating vortices. The velocity is given by:

\begin{equation}
\mathbf{v}(x, y, t) = \begin{pmatrix} -\partial_y \psi(x, y, t) \\ \partial_x \psi(\mathbf{x}, t) \end{pmatrix}, 
\end{equation}

where

\begin{align*}
\psi(x, y, t) &= \psi_0(x, y) + \psi_1(x, y, t) \\
\psi_0(x, y) &= c_3y-UL_y \tanh(\dfrac{y}{L_y}) \\
\psi_1(x,y,t) &= U L_ y\mathrm{sech}^2(\dfrac{y}{L_y}) \sum_{i=0}^{3} \epsilon_i f_i(x, t)e^{ik_ix}
\end{align*}

The external forcing term is given by:
\begin{equation}
f_i(x, t) = \cos(\dfrac{2i\pi}{L_x}(x-c_it))
\end{equation}

We have chosen the following set of parameters $$ c_1 = 0.1446U, c_2 = 0.205U, c_3 = 0.461U, \epsilon_1 = 0.0075, \epsilon_2 = 0.04, \epsilon_3 = 0.3, U = 62.66 (\dfrac{m}{s}), L_y = 1.77\times 10^6 (m), L_x = 6371\pi(km) $$

In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

def k(n, Len_X):
    
    return 2*n*np.pi/Len_X

def psi0(params, y):
    
    U = params[0]
    Len_Y = params[3]
    
    return -U*Len_Y*np.tanh(y/Len_Y)

def psi1(params, x, y, t):
    
    U = params[0]
    R = params[1]
    Len_X = params[2]
    len_Y = params[3]
    eps1 = params[4][0]
    eps2 = params[4][1]
    eps3 = params[4][2]
    
    c1 = 0.1446*U
    c2 = 0.205*U
    c3 = 0.461*U
    
    forcing3 = np.cos(k(3, Len_X)*(x-c3*t))
    forcing2 = np.cos(k(2, Len_X)*(x-c2*t))
    forcing1 = np.cos(k(1, Len_X)*(x-c1*t))
    
    return U*Len_Y*(sech(y/Len_Y)**2)*(eps1*forcing1+ eps2*forcing2 + eps3*forcing3)

def sech(x):
    
    return 2*np.exp(x)/(np.exp(2*x)+1)

def tanh(x):
    
    return (np.exp(2*x)-1)/(np.exp(2*x)+1)

def psi(params, x, y, t):

    return psi0(params, y)+psi1(params, x, y, t)

def Bickley_jet(params, x, y, t, dx, dy):
    
    X, Y, time = np.meshgrid(x, y, t)
    
    u = np.zeros(X.shape)
    v = np.zeros(Y.shape)
    
    for i in tqdm(range(X.shape[0])):
        
        for j in range(Y.shape[1]):
            
            for r in range(time.shape[2]):
                
                x_ = X[i, j, r]
                y_ = Y[i, j, r]
                t_ = time[i, j, r]
                
                #velx, vely = velocity(x_, y_, U, t_, eps, k, Len_X)
                
                u[i, j, r] = -(psi(params, x_, y_ + dy, t_)-psi(params, x_, y_- dy, t_))/(2*dy)
                v[i, j, r] = (psi(params, x_ + dx, y_, t_)-psi(params, x_ - dx, y_, t_))/(2*dx)
    
    return u, v

In [2]:
# U = 62.66*3600*24 (in meters/day)
U = 62.66*3600*24

# Radius of the Earth (in meters)
earthRadius = 6371*(10**3)

# L_x = pi*earthRadius
Len_X = np.pi*earthRadius

# L_y = 1.77*10**6 (in meters)
Len_Y = 1.77*(10**6)

# [epsilon_1, epsilon_2, epsilon_3]
epsilon = [.0075, .04, .3]

# time array
t = np.arange(0, 41, 1)
x = np.linspace(0, 20*Len_X, 400)
y = np.linspace(-2.25*Len_Y*1.25, 2.25*Len_Y*1.25, 20)

params = [U, earthRadius, Len_X, Len_Y, epsilon]

# Define grid size of mesh
dx = Len_X/5000
dy = Len_Y/5000

u, v = Bickley_jet(params, x, y, t, dx, dy)
            
scipy.io.savemat('./Bickley.mat', {'U': u, 'V': v, 'x': x, 'y': y, 't': t})

  0%|          | 0/20 [00:00<?, ?it/s]